<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
#default_exp trainer.base

In [ ]:
# export
import os
from pprint import pprint

import torch


class TTSTrainer:
    def __init__(self, hparams):
        self.hparams = hparams
        for k, v in hparams.values().items():
            setattr(self, k, v)

        if not hasattr(self, "debug"):
            self.debug = False
        if self.debug:
            print("Running in debug mode with hparams:")
            pprint(hparams.values())

    def save_checkpoint(self, checkpoint_name, **kwargs):
        checkpoint = {}
        for k, v in kwargs.items():
            if hasattr(v, "state_dict"):
                checkpoint[k] = v.state_dict()
            else:
                checkpoint[k] = v
        torch.save(
            checkpoint, os.path.join(self.checkpoint_path, f"{checkpoint_name}.pt")
        )

    def load_checkpoint(self, checkpoint_name):
        return torch.load(os.path.join(self.checkpoint_path, checkpoint_name))

    def train():
        raise NotImplemented
        # for batch in enumerate(data):
        #    #fill in

In [ ]:
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
hp = HParams(foo="bar", baz=123)
trainer = TTSTrainer(hp)
assert trainer.hparams == hp
assert trainer.foo == "bar"
assert trainer.baz == 123

In [ ]:
# export
from typing import List
import torch
from torch import nn
from torch.utils.data import DataLoader
from uberduck_ml_dev.data_loader import TextMelDataset, TextMelCollate
from uberduck_ml_dev.models.mellotron import Tacotron2


class Tacotron2Loss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, model_output: List, target: List):
        mel_target, gate_target = target[0], target[1]
        mel_target.requires_grad = False
        gate_target.requires_grad = False
        gate_target = gate_target.view(-1, 1)
        mel_out, mel_out_postnet, gate_out, _ = model_output
        gate_out = gate_out.view(-1, 1)
        mel_loss = nn.MSELoss()(mel_out, mel_target) + nn.MSELoss()(
            mel_out_postnet, mel_target
        )
        gate_loss = nn.BCEWithLogitsLoss()(gate_out, gate_target)
        return mel_loss + gate_loss


class MellotronTrainer(TTSTrainer):
    REQUIRED_HPARAMS = [
        "audiopaths_and_text",
        "checkpoint_path",
        "dataset_path",
        "epochs",
        "mel_fmax",
        "mel_fmin",
        "n_mel_channels",
        "text_cleaners",
    ]

    def validate(self, **kwargs):
        model = kwargs["model"]
        val_set = kwargs["val_set"]
        collate_fn = kwargs["collate_fn"]
        criterion = kwargs["criterion"]
        if self.distributed_run:
            raise NotImplemented
        total_loss = 0
        total_steps = 0
        with torch.no_grad():
            val_loader = DataLoader(
                val_set,
                shuffle=False,
                batch_size=self.batch_size,
                collate_fn=collate_fn,
            )
            for batch in val_loader:
                total_steps += 1
                X, y = model.parse_batch(batch)
                y_pred = model(X)
                loss = criterion(y_pred, y)
                total_loss += loss.item()
            mean_loss = total_loss / total_steps
            print(f"Average loss: {mean_loss}")

    @property
    def training_dataset_args(self):
        return [
            self.dataset_path,
            self.training_audiopaths_and_text,
            self.text_cleaners,
            # audio params
            self.n_mel_channels,
            self.sample_rate,
            self.mel_fmin,
            self.mel_fmax,
            self.filter_length,
            self.hop_length,
            self.win_length,
            self.max_wav_value,
            self.include_f0,
        ]

    @property
    def val_dataset_args(self):
        val_args = [a for a in self.training_dataset_args]
        val_args[1] = self.val_audiopaths_and_text
        return val_args

    def train(self):
        train_set = TextMelDataset(
            *self.training_dataset_args,
            debug=self.debug,
            debug_dataset_size=self.batch_size,
        )
        val_set = TextMelDataset(
            *self.val_dataset_args, debug=self.debug, debug_dataset_size=self.batch_size
        )
        collate_fn = TextMelCollate(n_frames_per_step=1, include_f0=self.include_f0)
        train_loader = DataLoader(
            train_set, batch_size=self.batch_size, shuffle=True, collate_fn=collate_fn
        )
        criterion = Tacotron2Loss()
        
        model = Tacotron2(self.hparams)
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=self.learning_rate,
            weight_decay=self.weight_decay,
        )
        start_epoch = 0
        if self.checkpoint_name:
            checkpoint = self.load_checkpoint(self.checkpoint_name)
            model.load_state_dict(checkpoint["model"])
            optimizer.load_state_dict(checkpoint["optimizer"])
            start_epoch = checkpoint["iteration"]
        # main training loop
        for epoch in range(start_epoch, self.epochs):
            for batch in train_loader:
                model.zero_grad()
                X, y = model.parse_batch(batch)
                y_pred = model(X)
                loss = criterion(y_pred, y)
                # TODO: fix for distributed run
                if self.distributed_run:
                    raise NotImplemented
                else:
                    reduced_loss = loss.item()
                loss.backward()
                if self.fp16_run:
                    raise NotImplemented
                else:
                    grad_norm = torch.nn.utils.clip_grad_norm(
                        model.parameters(), self.grad_clip_thresh
                    )
                optimizer.step()
                print(f"Loss: {reduced_loss}")
            if epoch % self.epochs_per_checkpoint == 0:
                self.save_checkpoint(
                    f"mellotron_{epoch}",
                    model=model,
                    optimizer=optimizer,
                    iteration=epoch,
                    learning_rate=self.learning_rate,
                )

            # There's no need to validate in debug mode since we're not really training.
            if self.debug:
                continue
            self.validate(
                model=model,
                val_set=val_set,
                collate_fn=collate_fn,
                criterion=criterion,
            )